In [16]:
import pickle
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from textblob import TextBlob, Word
from pandas import DataFrame
import re
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string

import gensim
from gensim.utils import simple_preprocess
import warnings
warnings.filterwarnings('ignore')


In [17]:
# read in pickle
with open('result2.pkl', 'rb') as f:
    result = pickle.load(f)

# convert to dataframe and store as pickle
df = pd.DataFrame(result)
df.to_pickle('df.pkl')

# print df head
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,86ff1ea1-0b63-43ce-addc-eb43f6193b3b,Yaseen Yaseen,https://play-lh.googleusercontent.com/a/ALm5wu...,Yaeen Yaeen gg,5,0,None,2022-10-04 20:32:28,None,NaT
1,3577f7a1-3394-4e77-813d-095a82cf8bcf,Kemar Richardson,https://play-lh.googleusercontent.com/a-/ACNPE...,Great,5,0,26.3.4,2022-10-04 20:32:10,None,NaT
2,7c8c56d9-d8ad-47d4-b24b-5289aa4529ff,Tracy Dunn,https://play-lh.googleusercontent.com/a/ALm5wu...,good,5,0,26.4.3,2022-10-04 20:31:21,None,NaT
3,80db804f-cccd-4b09-b690-abc12cbf0612,SG. Mugo. (Mugoz:),https://play-lh.googleusercontent.com/a-/ACNPE...,Good app,5,0,26.3.4,2022-10-04 20:30:22,None,NaT
4,4ed35e90-0f45-4865-81c4-b3a6f2ea49f7,Mwansa Judy,https://play-lh.googleusercontent.com/a-/ACNPE...,Most amazing app,5,0,26.3.4,2022-10-04 20:29:25,None,NaT


In [8]:
# check fields inside df
df.columns

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt'],
      dtype='object')

In [9]:
# check how many reviews with score of 1
df[df.score == 1].count()

reviewId                14178
userName                14178
userImage               14178
content                 14178
score                   14178
thumbsUpCount           14178
reviewCreatedVersion     9491
at                      14178
replyContent               13
repliedAt                  13
dtype: int64

We will implement functions to clean up the reviews, we will want to remove unnecesary stop words, punctuations @mentions, #hashtags, emojis, emails etc.
We will also perform english detection and spelling correction to ensure the correctedness of the cleaned data 

In [18]:

def remove_punctuations(review: str) -> str:
    words: list[str] = review.split()
    table: dict[int, int] = str.maketrans('', '', string.punctuation)
    stripped: list[str] = [w.translate(table) for w in words]
    return " ".join(stripped)

def normalize(review: str) -> str:
    return review.lower()

def remove_stopwords(review: str, stopwords: set[str]) -> str:
    words: list[str] = review.split()
    words = [w for w in words if not w in stopwords]
    return " ".join(words)


def stemming(review: str) -> str:
    tokens: list[str] = word_tokenize(review)
    porter: PorterStemmer = PorterStemmer()
    stemmed: list[str] = [porter.stem(word) for word in tokens]
    return " ".join(stemmed)

def lemmatize(review: str) -> str:
    lemmatizer = WordNetLemmatizer()
    tokens: list[str] = word_tokenize(review)
    lemmatized: list[str] = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(lemmatized)

def depure_data(data):
    
    #remove url with a RegEx
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    #remove email address
    data = re.sub('\S*@\S*\s?', '', data)

    #remove new line characters
    data = re.sub('\s+', ' ', data)

    #remove single quotes
    data = re.sub("\'", "", data)
        
    return data


def detokenize(text):
    #treebank tokenizer uses RegEx to tokenize text as in Penn Treebank
    return TreebankWordDetokenizer().detokenize(text)


def sent_to_words(sentence):
    return gensim.utils.simple_preprocess(str(sentence), deacc=True)
        
def correct_word_spelling(word):
    word = Word(word)
    result = word.correct()
    return result

def correct_sentence(sentence: list[str], spell):
    for j in range(len(sentence)):
        word = sentence[j]

        #use regex to reduce 3 consecutive letters to 2
        reduced_word = re.sub(r'(.)\1+', r'\1\1', word)
        sentence[j] = reduced_word

        #if incorrect spelled, correct word
        if sentence[j] != "tiktok" and len(spell.unknown([sentence[j]])) > 0:
            sentence[j] = correct_word_spelling(sentence[j])

    
    

In [19]:
def clean_review(review: str, stopwords: set[str]) -> str:
    result: str = depure_data(remove_stopwords(normalize(remove_punctuations(review)), stopwords))
    return result

In [28]:
def clean_data(df: DataFrame):
    # df['content'] = df['content'].apply(lambda x: x if is_english(x) else '')
    # df = df[df['content'] != '']
    spell = SpellChecker()
    review: list[str] = list(df["content"])
    stop_words: set[str] = set(stopwords.words('english'))
    cleaned_reviews: list[str] = list()
    for i, review in enumerate(review):
        cleaned_review = clean_review(review, stop_words)
        cleaned_review_in_words = sent_to_words(clean_review)
        correct_sentence(cleaned_review_in_words, spell)
        cleaned_review = detokenize(cleaned_review_in_words) 
        cleaned_reviews.append(cleaned_review)
        if i%10==0:
            print(i)
    df["Clean_texts"] = cleaned_review

In [29]:
clean_data(df)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030


KeyboardInterrupt: 

In [ ]:
df

In [9]:
# from langdetect import detect
# def detect_my(text):
#    try:
#        return detect(text)
#    except:
#        return 'unknown'

# # df['language'] = df['content'].apply(detect_my)

# detect_my('english')

In [10]:
import re

# return new column "is_english" with True or False
def is_english(text):
    try:
        return bool(re.search('[a-zA-Z]', text))
    except:
        return False
    

df['is_english'] = df['content'].apply(is_english)

# remove puncutation
df['content'] = df['content'].str.replace('[^\w\s]','')

# save new df as pickle
df.to_pickle('df_english.pkl')

# save df to json
df.to_json('df_english.json')

# save df to csv
df.to_csv('df_english.csv')

C:\Users\ngliw\AppData\Local\Temp\ipykernel_18336\165540621.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]','')
